In [ ]:
!pip install tensorflow
!pip install mne
!pip install gym
!pip install markupsafe==1.1.1
import tensorflow as tf
print('TF VERSION',tf.__version__)

# !pip uninstall tensorflow
# !pip uninstall tensorflow-gpu
# !pip install tensorflow==2.15.0
# !pip install tensorflow-gpu==2.15.0

In [ ]:
import scipy.io
import numpy as np
import mne
import os
import gym
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [ ]:
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.preprocessing import StandardScaler


class NDStandardScaler(TransformerMixin):
    def __init__(self, **kwargs):
        self._scaler = StandardScaler(copy=True, **kwargs)
        self._orig_shape = None

    def fit(self, X, **kwargs):
        X = np.array(X)
        # Save the original shape to reshape the flattened X later
        # back to its original shape
        if len(X.shape) > 1:
            self._orig_shape = X.shape[1:]
        X = self._flatten(X)
        self._scaler.fit(X, **kwargs)
        return self

    def transform(self, X, **kwargs):
        X = np.array(X)
        X = self._flatten(X)
        X = self._scaler.transform(X, **kwargs)
        X = self._reshape(X)
        return X

    def _flatten(self, X):
        # Reshape X to <= 2 dimensions
        if len(X.shape) > 2:
            n_dims = np.prod(self._orig_shape)
            X = X.reshape(-1, n_dims)
        return X

    def _reshape(self, X):
        # Reshape X back to it's original shape
        if len(X.shape) >= 2:
            X = X.reshape(-1, *self._orig_shape)
        return X

In [ ]:
import pywt
import scipy.signal
from scipy import stats
def mean(x):
    return np.mean(x, axis=-1).reshape(-1, 1)

def stddev(x):
    return np.std(x, axis=-1).reshape(-1, 1)

def peaktopeak(x):
    return np.ptp(x, axis=-1).reshape(-1, 1)

def variance(x):
    return np.var(x, axis=-1).reshape(-1, 1)

def mini(x):
    return np.min(x, axis=-1).reshape(-1, 1)

def maxi(x):
    return np.max(x, axis=-1).reshape(-1, 1)

def argmini(x):
    return np.argmin(x, axis=-1).reshape(-1, 1)

def argmaxi(x):
    return np.argmax(x, axis=-1).reshape(-1, 1)

def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1)).reshape(-1, 1)

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1).reshape(-1, 1)

def skewness(x):
    return stats.skew(x, axis=-1).reshape(-1, 1)

def kurtosis(x):
    return stats.kurtosis(x, axis=-1).reshape(-1, 1)

def concat_features(x):
    features = np.concatenate(
        (
            peaktopeak(x),
            rms(x),
            abs_diff_signal(x),
            skewness(x),
            kurtosis(x),
            variance(x),
            mean(x),
            stddev(x)
        ),
        axis=1
    )
    return features

def apply_cwt(data, scales, wavelet_name='morl'):
    """
    Apply Continuous Wavelet Transform (CWT) to EEG data.

    :param data: EEG data in CSP space with shape (components, timepoints)
    :param scales: Scales for CWT
    :param wavelet_name: Name of the mother wavelet for CWT
    :return: CWT coefficients
    """
    cwt_coeffs = np.array([pywt.cwt(data[i, :], scales, wavelet_name)[0] for i in range(data.shape[0])])
    return cwt_coeffs

    
def featuresarray_load(data_array):
    features = []
    fs = 500
    for d in data_array:
        
       
        alpha = mne.filter.filter_data(d, sfreq=fs, l_freq=8, h_freq=12,verbose=False)
        beta = mne.filter.filter_data(d, sfreq=fs, l_freq=12, h_freq=30,verbose=False)
        
        alph_ftrs = concat_features(alpha)
        beta_ftrs = concat_features(beta)
        
        #nperseg = 256
        
        
        _,p=scipy.signal.welch(beta, fs=fs,average='median',nfft = data_array.shape[2]//2)
        _,p2=scipy.signal.welch(alpha, fs=fs,average='median',nfft = data_array.shape[2]//2)
        

        res = np.mean([alph_ftrs,beta_ftrs],axis=0)
        #print('p',p.shape,res.shape)
        res = np.concatenate((res,p,p2),axis=1)
        #print(res.shape)
        features.append(res)
    return np.array(features)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from mne.decoding import CSP
import mne
from mne.decoding import CSP
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline

event_ids = [7,8,9,10]  
event_id_to_label = {}
for i in range(len(event_ids)):
    event_id_to_label[i] = event_ids[i]
print(event_id_to_label)
path = 'data/A09T.gdf'
raw = mne.io.read_raw_gdf(path, eog=['EOG-left', 'EOG-central', 'EOG-right'], preload=True)
raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])
ica = mne.preprocessing.ICA(n_components=len(raw.info['ch_names']), random_state=42, max_iter=1000)
ica.fit(raw)
ica.apply(raw)
csp_filters = {} 
events = mne.events_from_annotations(raw)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import stft
action_dict = {0:'left_hand',1:'right_hand',2:'foot',3:'tongue'}


# Create epochs for all events
all_epochs = mne.Epochs(raw, events[0], event_id=event_ids,  # No specific event_id filtering
                        tmin=1, tmax=6, baseline=None, preload=True)
all_epochs.pick_types(meg=False, eeg=True)

X_all = all_epochs.get_data()
event_ids_all = all_epochs.events[:, -1]
print(X_all.shape,event_ids_all.shape)
# sf = 250
# for trial in range(1):
#     for i in range(1):
#         # Compute the STFT

#         frequencies, times, Zxx = stft(X_all[trial,i,:], fs=sf, window='hann', nperseg=64, noverlap=0.5*64)

#         # Plot the spectrogram
#         plt.pcolormesh(times, frequencies, np.abs(Zxx), shading='gouraud')
#         plt.title(f'STFT Spectrogram of EEG Signal: label={action_dict[event_ids_all[trial]-7]}, channel={i}')
#         plt.ylabel('Frequency [Hz]')
#         plt.xlabel('Time [sec]')
#         plt.colorbar(label='Intensity')
        
#         plt.show()

'''# Loop over each event ID for OvR CSP
for event_id in event_ids:
    # Generate binary labels: current class (1) vs rest (0)
    y = (event_ids_all == event_id).astype(int)

    # Check if both classes are present
    if np.unique(y).size < 2:
        print(f"Skipping event_id {event_id} as it does not have two classes for CSP.")
        continue

    print('y is:', y)  # Check the binary labels

    # Apply CSP for the current binary classification
    csp = CSP(n_components=4, norm_trace=False)
    csp.fit(X_all, y)
    csp_filters[event_id] = csp
    X_csp = csp.transform(X_all)

    # Append the features
    combined_features.append(X_csp)

combined_features = np.concatenate(combined_features, axis=1) if combined_features else np.array([])'''
ncomp = 5
csp_transformed_data = {}
for event_id in event_ids:
    y = (event_ids_all == event_id).astype(int)
    if np.unique(y).size < 2:
        print(f"Skipping event_id {event_id}.")
        continue

    csp = CSP(n_components=ncomp, norm_trace=False, transform_into='csp_space')
    csp.fit(X_all, y)
    csp_transformed_data[event_id] = csp.transform(X_all)

print('CSP FILTERS DICT:',csp_transformed_data)
# Combine CSP features for each trial based on its label
n_trials = len(X_all)  # Number of trials
n_components = ncomp       # Number of CSP components (assuming 3 for this example)
n_time_points = csp_transformed_data[7].shape[2]   # Number of time points in the transformed CSP data

# Initialize the combined_features array to hold CSP features for all trials
combined_features = np.zeros((n_trials, n_components, n_time_points))

# Loop through each trial and assign the CSP-transformed data
for i, label in enumerate(event_ids_all):
    # Fetch the CSP features for the current trial and class label
    # Adjust the indexing based on how your labels and csp_transformed_data are structured
    csp_features_for_label = csp_transformed_data.get(label, None)

    # Check if the label exists in the dictionary and if the index is within bounds
    if csp_features_for_label is not None and i < len(csp_features_for_label):
        combined_features[i, :, :] = csp_features_for_label[i]

print(combined_features.shape)

print(combined_features.shape)
y = np.zeros((X_all.shape[0], len(event_ids)))  

for i, event_id in enumerate(event_ids):
    binary_labels = (event_ids_all == event_id).astype(int)
    y[:, i] = binary_labels  
print(y)

y_flattened = np.argmax(y, axis=1)
print(y_flattened)
print(y_flattened.tolist().count(0),y_flattened.tolist().count(1),y_flattened.tolist().count(2),y_flattened.tolist().count(3))
# clf = Pipeline([('scaler',StandardScaler()),('SVC', SVC())])
print('features shape: ',combined_features.shape,y_flattened.shape)
# scores = cross_val_score(clf, combined_features, y_flattened, cv=10, scoring='accuracy')
# print("Multiclass classification accuracy: %f" % scores.mean())

ftrs = featuresarray_load(combined_features)

print('features shape: ',ftrs.shape,y_flattened.shape)
X_train, X_test, y_train, y_test = train_test_split(ftrs, y_flattened, train_size=0.85, random_state=42, stratify=y_flattened)
scaler = NDStandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from keras.layers import PReLU, Conv1D, Dropout, SpatialDropout1D, MaxPooling1D, GlobalMaxPooling1D, Layer, AveragePooling1D, LSTM, Reshape, BatchNormalization
from keras.regularizers import l1_l2
# model = Sequential([
#     Dense(256, activation='relu'),
#     Dense(128, activation='tanh'),
#     Dropout(0.2),
#     Dense(len(event_ids), activation='softmax')  
# ])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
GLOBAL_SHAPE_LENGTH = ftrs.shape[2]
model = Sequential([
        Reshape((GLOBAL_SHAPE_LENGTH,ncomp)),
        BatchNormalization(),

        Conv1D(64, kernel_size=7),
        PReLU(),
        BatchNormalization(),

        MaxPooling1D(pool_size=2),
        SpatialDropout1D(0.1),

        Conv1D(128, kernel_size=5),
        BatchNormalization(),
        PReLU(),
        AveragePooling1D(pool_size=2),
        SpatialDropout1D(0.1),

        LSTM(128, activation='tanh', recurrent_regularizer=l1_l2(l1=0.01, l2=0.01),return_sequences=True),
        BatchNormalization(),
        GlobalMaxPooling1D(),
        BatchNormalization(),
        Dense(units=128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        BatchNormalization(),
        Dropout(0.1),
        Dense(units=64, activation='relu'),
        BatchNormalization(),
        Dropout(0.1),
        Dense(units=4, activation='softmax')
    ])

#X_train, X_test, y_train, y_test = train_test_split(combined_features, y_flattened, train_size=0.7, random_state=42, stratify=y)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=300, validation_split=0.1, batch_size=32)


In [ ]:
print(model.evaluate(X_test,y_test)[1]*100.00)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

for _ in range(25):

  sample_index_train = random.randint(0, len(X_train) - 1)
  train_sample = X_train[sample_index_train]
  train_label = y_train[sample_index_train]

  match_indices = np.where(y_test == train_label)[0]
  sample_index_test = random.choice(match_indices)
  test_sample = X_test[sample_index_test]


  feature_axis = np.arange(len(train_sample))


  plt.figure(figsize=(24, 6))

  # Plot train sample
  plt.subplot(1, 2, 1)
  plt.plot(feature_axis, train_sample)
  plt.title(f"Train Sample {sample_index_train}, Label {y_train[sample_index_train]}")
  plt.xlabel("Feature Index")
  plt.ylabel("Feature Value")
 
  plt.grid(True)

  # Plot test sample
  plt.subplot(1, 2, 2)
  plt.plot(feature_axis, test_sample)
  plt.title(f"Test Sample {sample_index_test}, Label {y_test[sample_index_test]}")
  plt.xlabel("Feature Index")
  plt.ylabel("Feature Value")
 
  plt.grid(True)

  plt.show()


print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
lst = y_train.tolist()
print(lst.count(0),lst.count(1),lst.count(2),lst.count(3),GLOBAL_SHAPE_LENGTH)
lst = y_test.tolist()
print(lst.count(0),lst.count(1),lst.count(2),lst.count(3),GLOBAL_SHAPE_LENGTH)

In [ ]:
import gym
import numpy as np
import random

class Plasticity(gym.Env):
  #dataset=(X_train, y_train)
    def __init__(self, images_per_episode=1, dataset=(X_train, y_train), random=True):
        super().__init__()

        self.action_space = gym.spaces.Discrete(4)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf,
                                                shape=(GLOBAL_SHAPE_LENGTH,ncomp),
                                                dtype=np.float32)
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0

    def step(self, action):
        done = False
        reward = self.calculate_reward(action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {}

    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y[next_obs_idx])
            obs = self.x[next_obs_idx]
        else:
            obs = self.x[self.dataset_idx]
            self.expected_action = int(self.y[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs

    def calculate_reward(self, action):
      if action == self.expected_action:
          reward = 1
      else:
          reward = 0

      return reward

   


In [ ]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Input
from tensorflow.keras.optimizers import Adam
tf.compat.v1.experimental.output_all_intermediates(True)

In [ ]:
env = Plasticity()
states = env.observation_space.shape
actions = env.action_space.n
print(env.action_space)
print(states,actions)

In [ ]:
from keras.layers import Conv1D, Dropout, AveragePooling1D, SpatialDropout1D, MaxPooling1D, BatchNormalization, LSTM, Flatten, Dense, PReLU, Reshape
from keras.models import Sequential
from keras.backend import clear_session
from keras.regularizers import l1_l2

def build_model(states, actions):
    model = Sequential([
        Reshape((GLOBAL_SHAPE_LENGTH,ncomp,),input_shape=(1,ncomp,GLOBAL_SHAPE_LENGTH)),
        BatchNormalization(),

        Conv1D(64, kernel_size=7),
        PReLU(),
        BatchNormalization(),

        MaxPooling1D(pool_size=2),
        SpatialDropout1D(0.1),

        Conv1D(128, kernel_size=5),
        BatchNormalization(),
        PReLU(),
        AveragePooling1D(pool_size=2),
        SpatialDropout1D(0.1),

        LSTM(128, activation='tanh', recurrent_regularizer=l1_l2(l1=0.01, l2=0.01),return_sequences=True),
        BatchNormalization(),
        GlobalMaxPooling1D(),
        BatchNormalization(),
        Dense(units=128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        BatchNormalization(),
        Dropout(0.1),
        Dense(units=64, activation='relu'),
        BatchNormalization(),
        Dropout(0.1),
        Dense(units=actions, activation='linear')
    ])
    return model


In [ ]:
import tensorflow as tf
#!pip install keras-rl2
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory
from keras import __version__
import tensorflow as tf

# To reset all information and start fresh, clear the current Keras session:
tf.keras.backend.clear_session()

#with strategy.scope():
model = build_model(states, actions)

model.build(input_shape=(1,*states))

model.summary()


In [ ]:
def build_agent(model, actions):
    policy = EpsGreedyQPolicy(eps=1.0)
    memory = SequentialMemory(limit=30000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-4)
    return dqn, policy

In [ ]:
dqn, policy = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.legacy.Adam(learning_rate=0.0005,decay=1e-3), metrics=['mse'])

In [ ]:
from rl.callbacks import Callback

#print(dqn.policy.eps)
class LossHistory(Callback):
    def __init__(self):
        self.losses = []

    def on_step_end(self, step, logs={}):
        self.losses.append(logs['metrics'][0])

class LossHistory2(Callback):
    def __init__(self):
        self.losses = []

    def on_step_end(self, step, logs={}):
        self.losses.append(logs['metrics'][1])

class RewardHistory(Callback):
    def on_train_begin(self, logs={}):
        self.rewards = []

    def on_episode_end(self, episode, logs={}):
        self.rewards.append(logs['episode_reward'])

class ExponentialDecayEpsilonCallback(Callback):
    def __init__(self, initial_epsilon, min_epsilon, decay_rate, decay_steps):
        self.epsilon = initial_epsilon
        self.min_epsilon = min_epsilon
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
        self.step_count = 0

    def on_step_end(self, step, logs={}):
        self.step_count += 1
        # if(self.epsilon <= 1e-10):
        #     self.epsilon=0.5
        self.epsilon = self.min_epsilon + (self.epsilon - self.min_epsilon) * np.exp(-self.step_count / self.decay_steps)
        self.model.policy.eps = max(self.epsilon, self.min_epsilon)
        
        if(self.step_count % 100 == 0): print(f" Epsilon: {self.epsilon}")


initial_epsilon = 1.0  
min_epsilon = 0.0  
decay_rate = 0.0001    
decay_steps = 100000   

epsilon_decay_cb = ExponentialDecayEpsilonCallback(initial_epsilon, min_epsilon, decay_rate, decay_steps)


loss_history = LossHistory()
loss_history2 = LossHistory2()
reward_history = RewardHistory()


#dqn.load_weights(prefix+'bciiv2a_dqn_weights.hdf5')
dqn.fit(env, nb_steps=2500, callbacks=[loss_history,loss_history2,reward_history,epsilon_decay_cb], verbose=1)

In [ ]:
dqn.policy.eps = 0
dqn.fit(env, nb_steps=250, callbacks=[loss_history,loss_history2,reward_history], verbose=1)
dqn.fit(env, nb_steps=250, callbacks=[loss_history,loss_history2,reward_history], verbose=1)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold


def dqn_eval(dqn_agent,d):
    attempts, correct = 0, 0
    labels = d[1]
    eenv = Plasticity(dataset=d, random=False)
    thing = 1
    y_predFull = []
    y_trueest = []
    total_reward = 0



    try:

        while True:

            if thing == 1:
                obs = eenv.reset()
                thing = 0
            done = False
            while not done:

                obs_reshaped = np.reshape(obs, (1,) + obs.shape)

                q_values = dqn_agent.compute_q_values(obs_reshaped)
                #print(q_values)

                action = np.argmax(q_values)

                #print('action: ', action, " - ", labels[attempts])
                y_predFull.append(action)
                y_trueest.append(labels[attempts])

                obs, rew, done, _ = eenv.step(action)
                total_reward += rew
                if done:
                    attempts += 1

    except StopIteration:
        print()
        print('Validation done...','total reward=',total_reward)

        y_predFull = np.array(y_predFull)
        y_trueest = np.array(y_trueest)

        cm = confusion_matrix(y_trueest, y_predFull)
        print(y_predFull)
        print(y_trueest)

        print("Confusion Matrix:\n", cm)

        report = classification_report(y_trueest, y_predFull,digits=4)
        print("Classification Report:\n", report)

        report = classification_report(y_trueest, y_predFull,output_dict=True)

        correct = sum(y_predFull == y_trueest)
        attempts = len(y_trueest)
        accuracy = (float(correct) / attempts)
        print('Validation done...')
        print('Accuracy: {:.2f}%'.format(accuracy*100))

        F1 = report['macro avg']['f1-score']
        precision = report['macro avg']['precision']
        recall = report['macro avg']['recall']

        print("F1 Score: {:.2f}%".format(F1*100))
        print("Precision: {:.2f}%".format(precision*100))
        print("Recall: {:.2f}%".format(recall*100))

    y_predFull = []
    y_trueest = []
    return accuracy, F1, precision, recall

dqn_eval(dqn,d=(X_train,y_train))
print('***************************************************************************************')
dqn_eval(dqn,d=(X_test,y_test))
print('***************************************************************************************')
import math
def dqn_eval2(thing,dataset):

    n_splits = 10

    mean_rewards_per_fold = []
    acc_per_fold = []
    f1_mean, p_mean, r_mean = [], [], []


    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # if flag:
    #   train = X_train
    #   test = y_train
    # else:
    #   train = X_test
    #   test = y_test
    train, test = dataset
   
    for train_index, test_index in skf.split(train,test):
        xtr, xte = train[train_index],train[test_index]
        ytr, yte = test[train_index], test[test_index]

        test_env = Plasticity(dataset = (xte, yte),random=True)
        
        scores = dqn.test(test_env, nb_episodes=10, visualize=False, verbose=0)
        acc, f1, p, r = dqn_eval(dqn,d=(xte, yte))
        
        mean_reward = np.mean(scores.history['episode_reward'])
        mean_rewards_per_fold.append(mean_reward)
        acc_per_fold.append(acc)
        f1_mean.append(f1)
        r_mean.append(r)
        p_mean.append(p)
        

    overall_mean_reward = np.mean(mean_rewards_per_fold)

    f1_avg, p_avg, r_avg = np.mean(f1_mean), np.mean(p_mean), np.mean(r_mean)
    print(f"Overall Mean Reward across all folds: {overall_mean_reward}")
    acc_avg = np.mean(acc_per_fold)
    print(f"Overall Mean Accuracy across all folds: {acc_avg * 100} %")
    print(f"Overall Mean F1 across all folds: {f1_avg * 100} %")
    print(f"Overall Mean Precision across all folds: {p_avg * 100} %")
    print(f"Overall Mean Recall across all folds: {r_avg * 100} %")

def dqn_eval3(dqn_agent, dataset):

    n_splits = 10

    mean_rewards_per_fold = []
    acc_per_fold = []
    f1_mean, p_mean, r_mean = [], [], []
    
    train,test=dataset

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    for train_index, test_index in skf.split(train,test):
        xtr, xte = train[train_index],train[test_index]
        ytr, yte = test[train_index], test[test_index]

        test_env = Plasticity(dataset = (xte, yte),random=True)
        
        scores = dqn.test(test_env, nb_episodes=10, visualize=False, verbose=0)
        acc, f1, p, r = dqn_eval(dqn,d=(xte, yte))
        
        mean_reward = np.mean(scores.history['episode_reward'])
        mean_rewards_per_fold.append(mean_reward)
        acc_per_fold.append(acc)
        f1_mean.append(f1)
        r_mean.append(r)
        p_mean.append(p)
        

    overall_mean_reward = np.mean(mean_rewards_per_fold)

    f1_avg, p_avg, r_avg = np.mean(f1_mean), np.mean(p_mean), np.mean(r_mean)
    print(f"Overall Mean Reward across all folds: {overall_mean_reward}")
    acc_avg = np.mean(acc_per_fold)
    print(f"Overall Mean Accuracy across all folds: {acc_avg * 100} %")
    print(f"Overall Mean F1 across all folds: {f1_avg * 100} %")
    print(f"Overall Mean Precision across all folds: {p_avg * 100} %")
    print(f"Overall Mean Recall across all folds: {r_avg * 100} %")
features_array, label_array = np.concatenate((X_train,X_test)),np.concatenate((y_train,y_test))
dqn_eval2(thing=True, dataset=(features_array,label_array))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Confusion matrix data
conf_matrix = np.array([[10,  0,  0,  0],
 [ 0, 11,  0,  0],
 [ 0,  1, 10,  0],
 [ 0,  0,  0, 11]])

# Normalize the confusion matrix row-wise for percentages
row_sums = conf_matrix.sum(axis=1)
conf_matrix_percent = conf_matrix / row_sums[:, np.newaxis] * 100

# Create an annotation matrix for displaying counts and percentages
annotations = np.empty_like(conf_matrix, dtype=object)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        annotations[i, j] = f"{conf_matrix[i, j]}\n{conf_matrix_percent[i, j]:.2f}%"

# Plotting
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_percent, annot=annotations, fmt="", cmap="Blues", cbar=True,
            xticklabels=[0, 1, 2, 3], yticklabels=[0, 1, 2, 3])
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix with Samples and Percentages')
plt.show()

In [ ]:
device_name = '/device:GPU:0'
#dqn_eval2(thing=True,flag=False)
# print(np.mean(scores.history['episode_reward'])*100,'%')
# plt.plot(scores.history['episode_reward'])
# plt.title('Testing Rewards per Episode')
# plt.xlabel('Episode')
# plt.ylabel('Cumulative Reward')
# plt.show()
# print(np.mean(scores.history['episode_reward'])*100,'%')
dqn.policy.eps = 0
# train_new = np.load(file='/content/drive/MyDrive/a/bciiv2a_test_features_A02E.gdf.npy')
# train_new_labels = train_dict['A02T.gdf']['labels']

#dqn_eval(dqn,d=(X_test[len(X_test)-25:],y_test[len(y_test)-25:]))
dqn_eval(dqn,d=(X_test,y_test))

dqn.policy.eps = 0
scores = dqn.test(Plasticity(images_per_episode=1,random=True,dataset=(X_test,y_test)), nb_episodes=10, visualize=False, action_repetition=1, verbose=1)
print(np.mean(scores.history['episode_reward'])*100.,'%')


def moving_average(data, window_size):
    return [np.mean(data[i:i+window_size]) for i in range(len(data) - window_size + 1)]
losses = loss_history.losses
losses2 = loss_history2.losses
smoothed_losses = moving_average(losses, window_size=50)
print(smoothed_losses)
plt.plot(smoothed_losses)
plt.title('Smoothed Training Loss per Step')
plt.xlabel('Step')
plt.xlim(left=0)
plt.ylabel('Smoothed Loss')
plt.grid(True)
plt.show()

smoothed_losses2 = moving_average(losses2, window_size=70)
print(smoothed_losses2)
plt.plot(smoothed_losses2)
plt.title('Smoothed Training Loss per Step')
plt.xlabel('Step')
plt.xlim(left=0)
plt.ylabel('Smoothed Loss')
plt.grid(True)
plt.show()

# plt.plot(moving_average(reward_history.rewards, window_size=70))
# plt.title('Training Rewards per Episode')
# plt.xlabel('Episode')
# plt.ylabel('Cumulative Reward')
# plt.show()

print("___________________________________________________________________________________________________________________________")